# Telecom X – Parte 2: Predicción de Cancelación (Churn)

Este notebook está **dividido en 4 secciones** para seguir el flujo solicitado:
1) **Preparación de datos** (carga CSV tratado, limpieza defensiva, one-hot si hace falta, split y desbalance)  
2) **Correlación y selección de variables** (matriz de correlación, top variables, mutual information)  
3) **Modelos predictivos** (Regresión Logística *con* normalización y Random Forest *sin* normalización; métricas y matrices de confusión)  
4) **Interpretación y conclusiones** (coeficientes LR, importancias RF, permutación opcional, resumen estratégico)


## 1) Preparación de datos


In [ ]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pandas.api.types import is_object_dtype, is_categorical_dtype

# Ruta del CSV tratado en la Parte 1 (ajústala si usaste otro nombre)
CANDIDATES = ["telecomx_ml_ready_numeric.csv", "telecomx_ml_ready.csv", "telecomx_model_ready.csv"]
df = None
for path in CANDIDATES:
    if os.path.exists(path):
        df = pd.read_csv(path)
        print(f"✔️ Cargado: {path}  shape={df.shape}")
        break
if df is None:
    raise FileNotFoundError("No se encontró un CSV tratado. Asegúrate de tener uno de: " + ", ".join(CANDIDATES))

# Asegurar objetivo binario 0/1
if 'abandono' not in df.columns:
    raise ValueError("No se encontró la columna 'abandono' en el CSV.")

if not set(pd.unique(df['abandono'])).issubset({0,1}):
    df['abandono'] = (df['abandono'].astype('string').str.strip().str.lower()
                      .map({'yes':1,'no':0}).astype('int8'))

# Eliminar identificadores únicos si aún existen
for c in ['id_cliente','customerID']:
    if c in df.columns:
        df = df.drop(columns=[c])

# Si quedan categóricas, aplicar one-hot (drop_first para evitar colinealidad)
cat_cols = [c for c in df.columns if c!='abandono' and (is_object_dtype(df[c]) or is_categorical_dtype(df[c]))]
if cat_cols:
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True, dtype='int8')

# Separar X, y (todo numérico)
y = df['abandono'].astype('int8')
X = df.drop(columns=['abandono'])

# Chequeos
assert not X.isna().any().any(), "Hay NaNs en X."
assert set(pd.unique(y)).issubset({0,1}), "La variable objetivo debe ser 0/1."

print("X shape:", X.shape, "| y shape:", y.shape)

# Desbalance de clases
counts = y.value_counts().sort_index()
props  = y.value_counts(normalize=True).sort_index()
print("\n=== Distribución de clases ===")
print(f"Activos (0): {counts.get(0,0)} ({props.get(0,0)*100:.2f}%)")
print(f"Churn (1):  {counts.get(1,0)} ({props.get(1,0)*100:.2f}%)")

# Gráfico simple de barras (una figura por gráfico)
plt.figure()
plt.bar(['Activos (0)','Churn (1)'], [counts.get(0,0), counts.get(1,0)])
plt.title('Distribución de clases (conteo)')
plt.ylabel('Número de clientes')
for i, v in enumerate([counts.get(0,0), counts.get(1,0)]):
    plt.text(i, v*0.98, f'{v}', ha='center', va='top')
plt.show()


## 2) Correlación y selección de variables


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif

# Matriz de correlación (solo numéricas)
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
corr = pd.concat([X[num_cols], y], axis=1).corr()

print("Top correlaciones con 'abandono':")
print(corr['abandono'].sort_values(ascending=False))

# Heatmap con matplotlib
fig, ax = plt.subplots(figsize=(10,7))
im = ax.imshow(corr, vmin=-1, vmax=1, cmap='coolwarm')
ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.index)))
ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticklabels(corr.index)
ax.set_title('Matriz de correlación (numéricas + objetivo)')
fig.colorbar(im, ax=ax, label='Correlación')
plt.tight_layout()
plt.show()

# Mutual Information (captura relaciones no lineales)
mi = mutual_info_classif(X[num_cols], y, random_state=42)
mi_s = pd.Series(mi, index=num_cols).sort_values(ascending=False)
print("\nTop 15 por Mutual Information:")
display(mi_s.head(15))

# Gráfico top-15 MI
top = mi_s.head(15).sort_values(ascending=True)
plt.figure(figsize=(8,6))
plt.barh(top.index, top.values)
plt.title('Top 15 - Mutual Information con "abandono"')
plt.tight_layout()
plt.show()


## 3) Modelos predictivos (con y sin normalización)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay

# Split 70/30 estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
print("Train:", X_train.shape, " Test:", X_test.shape)

# Modelo A: Regresión Logística (requiere normalización)
logreg = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=2000, class_weight='balanced', solver='liblinear'))
])

# Modelo B: Random Forest (no requiere normalización)
rf = Pipeline(steps=[
    ('model', RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced'))
])

# Entrenamiento
logreg.fit(X_train, y_train)
rf.fit(X_train, y_train)

def evaluar(model, Xtr, ytr, Xte, yte, name="modelo"):
    y_pred = model.predict(Xte)
    try:
        y_proba = model.predict_proba(Xte)[:,1]
        roc = roc_auc_score(yte, y_proba)
    except Exception:
        y_proba, roc = None, np.nan

    acc = accuracy_score(yte, y_pred)
    pre = precision_score(yte, y_pred, zero_division=0)
    rec = recall_score(yte, y_pred)
    f1  = f1_score(yte, y_pred)

    print(f"\n=== {name} ===")
    print(f"Accuracy:  {acc:.3f}")
    print(f"Precision: {pre:.3f}")
    print(f"Recall:    {rec:.3f}")
    print(f"F1-score:  {f1:.3f}")
    print(f"ROC AUC:   {roc:.3f}")

    # Matriz de confusión (una figura)
    cm = confusion_matrix(yte, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm)
    ax.set_title(f"Matriz de confusión — {name}")
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Real")
    for (i, j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha='center', va='center')
    plt.tight_layout()
    plt.show()

    # Curva ROC si hay proba
    if y_proba is not None:
        RocCurveDisplay.from_predictions(yte, y_proba, name=name)
        plt.title(f"ROC — {name}")
        plt.tight_layout()
        plt.show()

    print("\nReporte de clasificación (TEST):")
    print(classification_report(yte, y_pred, digits=3))

evaluar(logreg, X_train, y_train, X_test, y_test, name="Regresión Logística (scaled)")
evaluar(rf,     X_train, y_train, X_test, y_test, name="Random Forest (no-scale)")


## 4) Interpretación y conclusiones


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

feat_names = list(X_train.columns)

# 4.1 Coeficientes de Regresión Logística (features ya estandarizadas dentro del pipeline)
lr_coef = logreg.named_steps['model'].coef_[0]
coef_ser = pd.Series(lr_coef, index=feat_names, name='coef_lr')

print("Top 15 coeficientes (valor absoluto) — Regresión Logística:")
display(coef_ser.reindex(coef_ser.abs().sort_values(ascending=False).index).head(15))

top_lr = coef_ser.reindex(coef_ser.abs().sort_values(ascending=True).index).tail(15)
plt.figure(figsize=(8,6))
plt.barh(top_lr.index, top_lr.values)
plt.title("Regresión Logística — Top coeficientes (signo importa)")
plt.tight_layout()
plt.show()

# 4.2 Importancias de Random Forest
rf_imp = rf.named_steps['model'].feature_importances_
rf_ser = pd.Series(rf_imp, index=feat_names, name='imp_rf')

print("\nTop 15 importancias — Random Forest:")
display(rf_ser.sort_values(ascending=False).head(15))

top_rf = rf_ser.sort_values(ascending=True).tail(15)
plt.figure(figsize=(8,6))
plt.barh(top_rf.index, top_rf.values)
plt.title("Random Forest — Top importancias")
plt.tight_layout()
plt.show()

# 4.3 (Opcional) Importancia por permutación agnóstica al modelo (en TEST)
try:
    r_perm = permutation_importance(logreg, X_test, y_test, n_repeats=10, random_state=42, scoring='f1')
    perm_ser = pd.Series(r_perm.importances_mean, index=feat_names, name='perm_lr')
    print("\nTop 10 — Permutation Importance (LR, ΔF1 en test):")
    display(perm_ser.sort_values(ascending=False).head(10))
except Exception as e:
    print("Permutation importance (LR) no disponible:", e)

# 4.4 Mini-conclusiones automáticas (guía para el informe)
def resumen_top(series, k=5, desc="variable"):
    return [f"{i+1}. {name}" for i, name in enumerate(series.sort_values(ascending=False).head(k).index)]

resumen = {
    "LR_top_coef": resumen_top(coef_ser.abs(), 5, "coef_lr"),
    "RF_top_imp":  resumen_top(rf_ser, 5, "imp_rf"),
}
print("\n=== Resumen rápido para tu informe ===")
print("LR — variables con mayor |coef|:", *resumen['LR_top_coef'], sep="\n")
print("\nRF — variables con mayor importancia:", *resumen['RF_top_imp'], sep="\n")


### Conclusiones e Insights (para tu informe)

- **Modelos:** se entrenaron **Regresión Logística (con normalización)** y **Random Forest (sin normalización)**.  
- **Rendimiento:** compara Accuracy/Precision/Recall/F1/ROC AUC (ver celdas anteriores).  
- **Variables clave:** según *coeficientes* (LR) e *importancias* (RF), destacan las relacionadas con:
  - **Tipo de contrato** (e.g., *month-to-month*): mayor riesgo de churn.  
  - **Tenencia** (*meses_en_empresa*): mayor antigüedad reduce churn.  
  - **Cargos mensuales**: valores altos aumentan el riesgo si no hay valor percibido.  
  - **Cargos totales**: suele ser factor protector (relacionado a lealtad).  
  - **Método de pago** (p. ej., *electronic check*): tiende a asociarse con mayor churn.  

**Recomendaciones de retención:**
1. Incentivar **migración a contratos anual/bianual** para clientes mensuales.  
2. Enfocar **onboarding y soporte proactivo** en los primeros 6 meses.  
3. **Revisar planes con cargos altos**; ofrecer bundles/downgrades guiados.  
4. **Promover pagos automáticos** (tarjeta/transferencia).  
5. Ofrecer **servicios protectores** (seguridad online/soporte) a segmentos de riesgo.

> Nota: ajusta estas conclusiones con tus *tops* de coeficientes e importancias concretos de tus datos.
